In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#!pip install pingouin
import seaborn as sns
import pingouin as pg
from sklearn.preprocessing import MinMaxScaler

missing_values=["N/a","na",np.nan,"N/A"]  
df = pd.read_csv('/content/drive/MyDrive/UIUC_proA/Feedback.csv',na_values=missing_values,low_memory=False)
df.head() 
# print DataFrame

ModuleNotFoundError: ignored

In [ ]:

import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error

# Generate a 5000x100 matrix with random values between 0 and 1
data_matrix = np.array(df)

# Replace some random entries with np.nan to represent missing values
missing_ratio = 0.1
mask = np.random.choice([True, False], data_matrix.shape, p=[missing_ratio, 1 - missing_ratio])
data_matrix_missing = data_matrix.copy()
data_matrix_missing[mask] = np.nan

# Apply NMF
nmf = NMF(n_components=50)
W = nmf.fit_transform(np.nan_to_num(data_matrix_missing))
H = nmf.components_

# Reconstruct the completed matrix
completed_data = np.dot(W, H)

# Replace the known values with the original values in the completed matrix
completed_data_known = completed_data.copy()
rows, cols = np.where(~mask)
completed_data_known[rows, cols] = data_matrix[rows, cols]

# Calculate the Mean Squared Error
mse = mean_squared_error(data_matrix, completed_data_known)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.47029928758362666


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
### Tunned NMF
import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Generate a 5000x100 matrix with random values between 0 and 1
data_matrix = np.array(df)
# Replace some random entries with np.nan to represent missing values
missing_ratio = 0.1
mask = np.random.choice([True, False], data_matrix.shape, p=[missing_ratio, 1 - missing_ratio])
data_matrix_missing = data_matrix.copy()
data_matrix_missing[mask] = np.nan

# Define a custom scoring function for GridSearchCV
def nmf_mse_score(estimator, X):
    W = estimator.transform(X)
    H = estimator.components_
    completed_data = np.dot(W, H)
    completed_data_known = completed_data.copy()
    
    for i, j in zip(*np.where(~mask)):
        completed_data_known[i, j] = data_matrix[i, j]
    
    mse = mean_squared_error(data_matrix, completed_data_known)
    return -mse

# Search for the optimal number of components using GridSearchCV
param_grid = {"n_components": list([16,32,64,99])}
nmf = NMF()
grid_search = GridSearchCV(nmf, param_grid, scoring=nmf_mse_score, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(np.nan_to_num(data_matrix_missing))

# Extract the best NMF model and number of components
best_nmf = grid_search.best_estimator_
best_n_components = grid_search.best_params_["n_components"]
print("Best number of components:", best_n_components)

# Reconstruct the completed matrix using the best NMF model
W = best_nmf.transform(np.nan_to_num(data_matrix_missing))
H = best_nmf.components_
completed_data = np.dot(W, H)

# Replace the known values with the original values in the completed matrix
completed_data_known = completed_data.copy()
for i, j in zip(*np.where(~mask)):
    completed_data_known[i, j] = data_matrix[i, j]

# Calculate the Mean Squared Error
mse = mean_squared_error(data_matrix, completed_data_known)
print("Mean Squared Error:", mse)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/n2/72wbbc713654l9zb1f35l2940000gn/T/ipykernel_30416/785363810.py", line 24, in nmf_mse_score
IndexError: index 1000 is out of bounds for axis 0 with size 1000

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/

Best number of components: 16
Mean Squared Error: 0.15918272407185813


In [ ]:
### Tunned NMF

import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Generate a 5000x100 matrix with random values between 0 and 1
data_matrix = np.array(df)
# Replace some random entries with np.nan to represent missing values
missing_ratio = 0.1
mask = np.random.choice([True, False], data_matrix.shape, p=[missing_ratio, 1 - missing_ratio])
data_matrix_missing = data_matrix.copy()
data_matrix_missing[mask] = np.nan

# Define a custom scoring function for GridSearchCV
def nmf_mse_score(estimator, X):
    W = estimator.transform(X)
    H = estimator.components_
    completed_data = np.dot(W, H)
    completed_data_known = completed_data.copy()
    
    for i, j in zip(*np.where(~mask)):
        completed_data_known[i, j] = data_matrix[i, j]
    
    mse = mean_squared_error(data_matrix, completed_data_known)
    return -mse

# Search for the optimal number of components using GridSearchCV
param_grid = {"n_components": list([16,32,64,99])}
nmf = NMF()
grid_search = GridSearchCV(nmf, param_grid, scoring=nmf_mse_score, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(np.nan_to_num(data_matrix_missing))

# Extract the best NMF model and number of components
best_nmf = grid_search.best_estimator_
best_n_components = grid_search.best_params_["n_components"]
print("Best number of components:", best_n_components)

# Reconstruct the completed matrix using the best NMF model
W = best_nmf.transform(np.nan_to_num(data_matrix_missing))
H = best_nmf.components_
completed_data = np.dot(W, H)

# Replace the known values with the original values in the completed matrix
completed_data_known = completed_data.copy()
for i, j in zip(*np.where(~mask)):
    completed_data_known[i, j] = data_matrix[i, j]

# Calculate the Mean Squared Error
mse = mean_squared_error(data_matrix, completed_data_known)
print("Mean Squared Error:", mse)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/n2/72wbbc713

Best number of components: 16
Mean Squared Error: 0.1582975658484862


In [ ]:
def rmse(df1, df2):
    return np.sqrt(((df1 - df2) ** 2).mean().mean())

In [ ]:
df_raw= pd.read_csv('/Users/cathyy/Desktop/UIUC/stat542/Projects/jester_data_NaN.csv',na_values=missing_values,low_memory=False)
df_raw.head()

,j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,...,j91,j92,j93,j94,j95,j96,j97,j98,j99,j100
0,2.160973,1.658903,3.654244,3.059159,2.461180,2.373188,1.281056,3.429089,2.864907,2.939959,...,1.813700,2.127660,3.190994,3.291925,3.240166,3.336793,2.745200,3.454969,3.002070,1.519151
1,3.103002,0.137164,4.006211,1.587961,0.375259,3.566253,1.783126,0.427019,3.454969,1.381988,...,3.274520,2.908666,3.768116,3.051242,3.051242,3.085106,2.179554,3.605072,2.650104,2.424948
2,4.006211,0.225155,2.701863,1.359626,2.298137,3.315217,0.188923,1.645963,3.403209,2.877847,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.474120,NaN,0.665114,1.506211,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.721014,NaN,NaN
4,3.731884,4.334886,4.697205,4.369486,4.270186,4.761905,4.736025,4.270186,0.540890,2.663043,...,0.869227,1.686559,4.133023,2.034161,4.798137,1.637260,1.421899,3.755176,0.137164,4.824017


In [ ]:
rmse_y = rmse(df, completed_data_known)

print(rmse_y)

0.39786626628615573


In [ ]:
# Calculate on x
rmse = rmse(df_raw, completed_data_known)

print(rmse)

0.4356655186479535


In [ ]:
import pandas as pd

# Replace missing values in all columns with the column-wise median
df_median = df_raw.fillna(df_raw.median())


In [ ]:
### Tunned NMF

import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Generate a 5000x100 matrix with random values between 0 and 1
data_matrix = np.array(df_median)
# Replace some random entries with np.nan to represent missing values
missing_ratio = 0.1
mask = np.random.choice([True, False], data_matrix.shape, p=[missing_ratio, 1 - missing_ratio])
data_matrix_missing = data_matrix.copy()
data_matrix_missing[mask] = np.nan

# Define a custom scoring function for GridSearchCV
def nmf_mse_score(estimator, X):
    W = estimator.transform(X)
    H = estimator.components_
    completed_data = np.dot(W, H)
    completed_data_known = completed_data.copy()
    
    for i, j in zip(*np.where(~mask)):
        completed_data_known[i, j] = data_matrix[i, j]
    
    mse = mean_squared_error(data_matrix, completed_data_known)
    return -mse

# Search for the optimal number of components using GridSearchCV
param_grid = {"n_components": list([16,32,64,99])}
nmf = NMF()
grid_search = GridSearchCV(nmf, param_grid, scoring=nmf_mse_score, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(np.nan_to_num(data_matrix_missing))

# Extract the best NMF model and number of components
best_nmf = grid_search.best_estimator_
best_n_components = grid_search.best_params_["n_components"]
print("Best number of components:", best_n_components)

# Reconstruct the completed matrix using the best NMF model
W = best_nmf.transform(np.nan_to_num(data_matrix_missing))
H = best_nmf.components_
completed_data = np.dot(W, H)

# Replace the known values with the original values in the completed matrix
completed_data_known_2 = completed_data.copy()
for i, j in zip(*np.where(~mask)):
    completed_data_known_2[i, j] = data_matrix[i, j]

# Calculate the Mean Squared Error
mse = mean_squared_error(data_matrix, completed_data_known_2)
print("Mean Squared Error:", mse)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packa

Best number of components: 16
Mean Squared Error: 0.15768567089064905


In [ ]:
def rmse(df1, df2):
    return np.sqrt(((df1 - df2) ** 2).mean().mean())

In [ ]:
# Calculate on x for median dataset
rmse = rmse(df_raw, completed_data_known_2)

print(rmse)

0.4318782130310835


In [ ]:
import pandas as pd

# Replace missing values in all columns with the column-wise minimum
df_min = df_raw.fillna(df_raw.min())
df_min.head()

,j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,...,j91,j92,j93,j94,j95,j96,j97,j98,j99,j100
0,2.160973,1.658903,3.654244,3.059159,2.461180,2.373188,1.281056,3.429089,2.864907,2.939959,...,1.813700,2.127660,3.190994,3.291925,3.240166,3.336793,2.745200,3.454969,3.002070,1.519151
1,3.103002,0.137164,4.006211,1.587961,0.375259,3.566253,1.783126,0.427019,3.454969,1.381988,...,3.274520,2.908666,3.768116,3.051242,3.051242,3.085106,2.179554,3.605072,2.650104,2.424948
2,4.006211,0.225155,2.701863,1.359626,2.298137,3.315217,0.188923,1.645963,3.403209,2.877847,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,2.474120,0.000000,0.665114,1.506211,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.721014,0.000000,0.000000
4,3.731884,4.334886,4.697205,4.369486,4.270186,4.761905,4.736025,4.270186,0.540890,2.663043,...,0.869227,1.686559,4.133023,2.034161,4.798137,1.637260,1.421899,3.755176,0.137164,4.824017


In [ ]:
### Tunned NMF

import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Generate a 5000x100 matrix with random values between 0 and 1
data_matrix = np.array(df_min)
# Replace some random entries with np.nan to represent missing values
missing_ratio = 0.1
mask = np.random.choice([True, False], data_matrix.shape, p=[missing_ratio, 1 - missing_ratio])
data_matrix_missing = data_matrix.copy()
data_matrix_missing[mask] = np.nan

# Define a custom scoring function for GridSearchCV
def nmf_mse_score(estimator, X):
    W = estimator.transform(X)
    H = estimator.components_
    completed_data = np.dot(W, H)
    completed_data_known = completed_data.copy()
    
    for i, j in zip(*np.where(~mask)):
        completed_data_known[i, j] = data_matrix[i, j]
    
    mse = mean_squared_error(data_matrix, completed_data_known)
    return -mse

# Search for the optimal number of components using GridSearchCV
param_grid = {"n_components": list([16,32,64,99])}
nmf = NMF()
grid_search = GridSearchCV(nmf, param_grid, scoring=nmf_mse_score, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(np.nan_to_num(data_matrix_missing))

# Extract the best NMF model and number of components
best_nmf = grid_search.best_estimator_
best_n_components = grid_search.best_params_["n_components"]
print("Best number of components:", best_n_components)

# Reconstruct the completed matrix using the best NMF model
W = best_nmf.transform(np.nan_to_num(data_matrix_missing))
H = best_nmf.components_
completed_data = np.dot(W, H)

# Replace the known values with the original values in the completed matrix
completed_data_known_3 = completed_data.copy()
for i, j in zip(*np.where(~mask)):
    completed_data_known_3[i, j] = data_matrix[i, j]

# Calculate the Mean Squared Error
mse = mean_squared_error(data_matrix, completed_data_known_3)
print("Mean Squared Error:", mse)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/P

Best number of components: 16
Mean Squared Error: 0.16925037870417703


In [ ]:
def rmse(df1, df2):
    return np.sqrt(((df1 - df2) ** 2).mean().mean())

In [ ]:
# Calculate on x for min dataset
rmse = rmse(df_raw, completed_data_known_3)

print(rmse)

0.47726840696888995


In [ ]:
import pandas as pd

# Replace missing values in all columns with zero
df_zero = df.fillna(0)
df_zero.head()


,j1,j2,j3,j4,j5,j6,j7,j8,j9,j10,...,j91,j92,j93,j94,j95,j96,j97,j98,j99,j100
0,2.160973,1.658903,3.654244,3.059159,2.461180,2.373188,1.281056,3.429089,2.864907,2.939959,...,1.813700,2.127660,3.190994,3.291925,3.240166,3.336793,2.745200,3.454969,3.002070,1.519151
1,3.103002,0.137164,4.006211,1.587961,0.375259,3.566253,1.783126,0.427019,3.454969,1.381988,...,3.274520,2.908666,3.768116,3.051242,3.051242,3.085106,2.179554,3.605072,2.650104,2.424948
2,4.006211,0.225155,2.701863,1.359626,2.298137,3.315217,0.188923,1.645963,3.403209,2.877847,...,3.103198,2.898042,3.213038,2.908229,2.844325,2.983153,3.038803,2.766359,2.563586,2.884340
3,2.833149,2.620653,2.666855,2.187868,2.474120,2.999772,0.665114,1.506211,2.452390,2.926614,...,3.103198,2.898042,3.213038,2.908229,2.844325,2.983153,3.038803,1.721014,2.563586,2.884340
4,3.731884,4.334886,4.697205,4.369486,4.270186,4.761905,4.736025,4.270186,0.540890,2.663043,...,0.869227,1.686559,4.133023,2.034161,4.798137,1.637260,1.421899,3.755176,0.137164,4.824017


In [ ]:
### Tunned NMF

import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Generate a 5000x100 matrix with random values between 0 and 1
data_matrix = np.array(df_zero)
# Replace some random entries with np.nan to represent missing values
missing_ratio = 0.1
mask = np.random.choice([True, False], data_matrix.shape, p=[missing_ratio, 1 - missing_ratio])
data_matrix_missing = data_matrix.copy()
data_matrix_missing[mask] = np.nan

# Define a custom scoring function for GridSearchCV
def nmf_mse_score(estimator, X):
    W = estimator.transform(X)
    H = estimator.components_
    completed_data = np.dot(W, H)
    completed_data_known = completed_data.copy()
    
    for i, j in zip(*np.where(~mask)):
        completed_data_known[i, j] = data_matrix[i, j]
    
    mse = mean_squared_error(data_matrix, completed_data_known)
    return -mse

# Search for the optimal number of components using GridSearchCV
param_grid = {"n_components": list([16,32,64,99])}
nmf = NMF()
grid_search = GridSearchCV(nmf, param_grid, scoring=nmf_mse_score, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(np.nan_to_num(data_matrix_missing))

# Extract the best NMF model and number of components
best_nmf = grid_search.best_estimator_
best_n_components = grid_search.best_params_["n_components"]
print("Best number of components:", best_n_components)

# Reconstruct the completed matrix using the best NMF model
W = best_nmf.transform(np.nan_to_num(data_matrix_missing))
H = best_nmf.components_
completed_data = np.dot(W, H)

# Replace the known values with the original values in the completed matrix
completed_data_known_4 = completed_data.copy()
for i, j in zip(*np.where(~mask)):
    completed_data_known_4[i, j] = data_matrix[i, j]

# Calculate the Mean Squared Error
mse = mean_squared_error(data_matrix, completed_data_known_4)
print("Mean Squared Error:", mse)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/n2/72wbbc713

Best number of components: 16
Mean Squared Error: 0.15800879840205528


In [ ]:
def rmse(df1, df2):
    return np.sqrt(((df1 - df2) ** 2).mean().mean())

In [ ]:
# Calculate on x for min dataset
rmse = rmse(df_raw, completed_data_known_4)

print(rmse)

0.4348509343035095
